# Code Snippet to investigate the text encoder:

In [1]:
import torch
import clip

# Check if the environment is using CUDA
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load the model and preprocess
model, preprocess = clip.load("ViT-B/32", device)

print("CLIP model loaded successfully!")

Using device: cpu


100%|███████████████████████████████████████| 338M/338M [01:37<00:00, 3.65MiB/s]


CLIP model loaded successfully!


In [ ]:
def word_encoder(words: list) -> dict[str, torch.Tensor]:
    embeddings = {}
    for word in words:
        tokenizer = clip.tokenize([word]).to(device)

        with torch.no_grad():
            embedding = model.encode_text(tokenizer)
        
        embeddings[word] = embedding

    return embeddings

def embedding_similarities(target_word, candidates):
    word_similarities = []
    target_embedding = next(iter(target_word.values())) 
    target_word_str = next(iter(target_word.keys())) 

    for word, embedding in candidates.items():
        similarity = torch.cosine_similarity(target_embedding, embedding)
        word_similarities.append((word, similarity.item()))

    word_similarities = sorted(word_similarities, key=lambda x: x[1], reverse=True)
    print(f"Comparing '{target_word_str}' with:")
    for word, similarity in word_similarities:
        print(f"{word}: {similarity:.4f}")


Word similarities:

In [ ]:
words = ["queen", "man", "woman"]
target = ["king"]

word_embed = word_encoder(words)
target_embed = word_encoder(target)

embedding_similarities(target_embed, word_embed)


Comparing 'king' with:
queen: 0.9483
woman: 0.8910
man: 0.8831


Vector analogies:

In [ ]:
words = ["queen", "prince", "child", "hero", "boss", "man", "woman"]
target = ["king"]
word_embed = word_encoder(words)
target_embed = word_encoder(target)

arithmetic = target_embed["king"] - word_embed["man"] + word_embed["woman"]
target_embed = {"Queen_approx": arithmetic}

embedding_similarities(target_embed, word_embed)


Comparing 'Queen_approx' with:
queen: 0.9025
woman: 0.8900
prince: 0.8650
boss: 0.8626
child: 0.8201
hero: 0.7572
man: 0.6932
